<a href="https://colab.research.google.com/github/Sreelakshmim6/CE888/blob/main/Assignment1_Project-2_TweetEval/TweetEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers


     |████████████████████████████████| 1.9MB 5.4MB/s 
     |████████████████████████████████| 3.2MB 22.7MB/s 
     |████████████████████████████████| 890kB 39.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=6ffb0e2436f6891cd87d67c261cb44e4067f44a33ec9d8ef92b23960eb4df1b9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
pip install tensorflow


In [11]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
#from transformers import BertTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)



MODEL = f"cardiffnlp/twitter-roberta-base-hate"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)



**Hate Speech Detection**

In [42]:
import pandas as pd
Hate_Train_labels = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/train_labels.txt"
with urllib.request.urlopen(Hate_Train_labels) as f:   
    html_1 = f.read().decode('utf-8').split("\n")
    
Hate_Train_labels_df=pd.DataFrame(html_1)
print(Hate_Train_labels_df)

Hate_Train_text = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/train_text.txt"
with urllib.request.urlopen(Hate_Train_text) as f:   
    html_2 = f.read().decode('utf-8').split("\n")
       
Hate_Train_text_df=pd.DataFrame(html_2)
print(Hate_Train_text_df)

      0
0     0
1     1
2     1
3     1
4     0
...  ..
8996  0
8997  1
8998  0
8999  1
9000   

[9001 rows x 1 columns]
                                                      0
0     @user nice new signage. Are you not concerned ...
1     A woman who you fucked multiple times saying y...
2     @user @user real talk do you have eyes or were...
3     your girlfriend lookin at me like a groupie in...
4                          Hysterical woman like @user 
...                                                 ...
8996  @user Good Luck @user More Americans #WalkAway...
8997            Bitch you can't keep up so stop trying 
8998  @user @user @user @user @user @user Japan is a...
8999  @user hi Best l see someone fucking girl comme...
9000                                                   

[9001 rows x 1 columns]


In [43]:
import pandas as pd
Hate_Test_labels = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/test_labels.txt"
with urllib.request.urlopen(Hate_Test_labels) as f:   
    html_1 = f.read().decode('utf-8').split("\n")
    
Hate_Test_label_df=pd.DataFrame(html_1)
print(Hate_Test_label_df)

Hate_Test_text = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/test_text.txt"
with urllib.request.urlopen(Hate_Test_text) as f:   
    html_2 = f.read().decode('utf-8').split("\n")
       
Hate_Test_text_df=pd.DataFrame(html_2)
print(Hate_Test_text_df)

      0
0     0
1     1
2     1
3     1
4     1
...  ..
2966  0
2967  0
2968  1
2969  1
2970   

[2971 rows x 1 columns]
                                                      0
0     @user , you are correct that Reid certainly is...
1               Whoever just unfollowed me you a bitch 
2     @user @user Those People Invaded Us!!! They DO...
3     stop JUDGING bitches by there cover, jus cuz s...
4     how about i knock heads off and send them gift...
...                                                 ...
2966  Silly Killary WANNABE !! And @user numbers JUS...
2967  @user @user @user @user @user @user @user @use...
2968                             @user StopImmigration 
2969  That bitch on the spoiled whore list   Thought...
2970                                                   

[2971 rows x 1 columns]


**Extracting Hashtags**

In [44]:
import re

#extract hashtags and put them in new column named hashtag 
Hate_Train_text_df["hashtags"]=Hate_Train_text_df[0].apply(lambda x:re.findall(r"#(\w+)",x))

Hate_Test_text_df["hashtags"]=Hate_Test_text_df[0].apply(lambda x:re.findall(r"#(\w+)",x))

In [45]:
Hate_Train_text_df

,0,hashtags
0,@user nice new signage. Are you not concerned ...,[]
1,A woman who you fucked multiple times saying y...,[]
2,@user @user real talk do you have eyes or were...,[]
3,your girlfriend lookin at me like a groupie in...,[]
4,Hysterical woman like @user,[]
...,...,...
8996,@user Good Luck @user More Americans #WalkAway...,"[WalkAwayFromDemocrat, WallkAway, Immigration,..."
8997,Bitch you can't keep up so stop trying,[]
8998,@user @user @user @user @user @user Japan is a...,[]
8999,@user hi Best l see someone fucking girl comme...,[]


**Translate Emoji**

In [21]:
#conda install -c conda-forge emoji
!pip install unicode



In [25]:
pip install emoji --upgrade


     |████████████████████████████████| 133kB 5.6MB/s 


In [46]:
#preprocessing tweets training and testing
from emoji import UNICODE_EMOJI
import emoji

#translate emojis in training
Hate_Train_text_df["clean"]=Hate_Train_text_df[0].apply(lambda x: emoji.demojize(x))

Hate_Test_text_df["clean"]=Hate_Test_text_df[0].apply(lambda x: emoji.demojize(x))

In [47]:
Hate_Train_text_df


,0,hashtags,clean
0,@user nice new signage. Are you not concerned ...,[],@user nice new signage. Are you not concerned ...
1,A woman who you fucked multiple times saying y...,[],A woman who you fucked multiple times saying y...
2,@user @user real talk do you have eyes or were...,[],@user @user real talk do you have eyes or were...
3,your girlfriend lookin at me like a groupie in...,[],your girlfriend lookin at me like a groupie in...
4,Hysterical woman like @user,[],Hysterical woman like @user
...,...,...,...
8996,@user Good Luck @user More Americans #WalkAway...,"[WalkAwayFromDemocrat, WallkAway, Immigration,...",@user Good Luck @user More Americans #WalkAway...
8997,Bitch you can't keep up so stop trying,[],Bitch you can't keep up so stop trying
8998,@user @user @user @user @user @user Japan is a...,[],@user @user @user @user @user @user Japan is a...
8999,@user hi Best l see someone fucking girl comme...,[],@user hi Best l see someone fucking girl comme...


**Removing URLs**

In [48]:
#remove urls 
Hate_Train_text_df["clean"]=Hate_Train_text_df["clean"].apply(lambda x: re.sub(r"http:\S+",'',x))

Hate_Test_text_df["clean"]=Hate_Test_text_df["clean"].apply(lambda x: re.sub(r"http:\S+",'',x))

In [49]:
Hate_Train_text_df

,0,hashtags,clean
0,@user nice new signage. Are you not concerned ...,[],@user nice new signage. Are you not concerned ...
1,A woman who you fucked multiple times saying y...,[],A woman who you fucked multiple times saying y...
2,@user @user real talk do you have eyes or were...,[],@user @user real talk do you have eyes or were...
3,your girlfriend lookin at me like a groupie in...,[],your girlfriend lookin at me like a groupie in...
4,Hysterical woman like @user,[],Hysterical woman like @user
...,...,...,...
8996,@user Good Luck @user More Americans #WalkAway...,"[WalkAwayFromDemocrat, WallkAway, Immigration,...",@user Good Luck @user More Americans #WalkAway...
8997,Bitch you can't keep up so stop trying,[],Bitch you can't keep up so stop trying
8998,@user @user @user @user @user @user Japan is a...,[],@user @user @user @user @user @user Japan is a...
8999,@user hi Best l see someone fucking girl comme...,[],@user hi Best l see someone fucking girl comme...


**Tokenizing**

In [51]:
import nltk
nltk.download('punkt')

#tokenize  tweets
Hate_Train_text_df["clean"]=Hate_Train_text_df["clean"].apply(lambda x: nltk.word_tokenize(str(x).lower()))

Hate_Test_text_df["clean"]=Hate_Test_text_df["clean"].apply(lambda x: nltk.word_tokenize(str(x).lower()))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [52]:
Hate_Test_text_df

,0,hashtags,clean
0,"@user , you are correct that Reid certainly is...","[MAGA, KAG, POTUS, Trump, NEWS, VoteRed, NoDAC...","[@, user, ,, you, are, correct, that, reid, ce..."
1,Whoever just unfollowed me you a bitch,[],"[whoever, just, unfollowed, me, you, a, bitch]"
2,@user @user Those People Invaded Us!!! They DO...,"[AmericaFIRST, Christian, NODACA]","[@, user, @, user, those, people, invaded, us,..."
3,"stop JUDGING bitches by there cover, jus cuz s...",[],"[stop, judging, bitches, by, there, cover, ,, ..."
4,how about i knock heads off and send them gift...,[],"[how, about, i, knock, heads, off, and, send, ..."
...,...,...,...
2966,Silly Killary WANNABE !! And @user numbers JUS...,"[MAGA, MASA, BuildThatWall, BuildTheFuture]","[silly, killary, wannabe, !, !, and, @, user, ..."
2967,@user @user @user @user @user @user @user @use...,"[IllegalImmigrants, VoteBank]","[@, user, @, user, @, user, @, user, @, user, ..."
2968,@user StopImmigration,[],"[@, user, stopimmigration]"
2969,That bitch on the spoiled whore list Thought...,[],"[that, bitch, on, the, spoiled, whore, list, t..."


**Stopwords Removal**

In [54]:
#import stopwords corpus from nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
#import punctuation charachers
import string 
#remove stopwords and punctuations
stop_words = set(stopwords.words('english'))
Hate_Train_text_df["clean"]=Hate_Train_text_df["clean"].apply(lambda x: [y for y in x if (y not in stop_words)])
Hate_Train_text_df["clean"]=Hate_Train_text_df["clean"].apply(lambda x: [re.sub(r'['+string.punctuation+']','',y) for y in x])
Hate_Train_text_df["clean"]=Hate_Train_text_df["clean"].apply(lambda x: [re.sub('\\n','',y) for y in x])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [56]:
Hate_Test_text_df["clean"]=Hate_Test_text_df["clean"].apply(lambda x: [y for y in x if (y not in stop_words)])
Hate_Test_text_df["clean"]=Hate_Test_text_df["clean"].apply(lambda x: [re.sub(r'['+string.punctuation+']','',y) for y in x])
Hate_Test_text_df["clean"]=Hate_Test_text_df["clean"].apply(lambda x: [re.sub('\\n','',y) for y in x])

In [57]:
Hate_Test_text_df

,0,hashtags,clean
0,"@user , you are correct that Reid certainly is...","[MAGA, KAG, POTUS, Trump, NEWS, VoteRed, NoDAC...","[, user, , correct, reid, certainly, weasel, ,..."
1,Whoever just unfollowed me you a bitch,[],"[whoever, unfollowed, bitch]"
2,@user @user Those People Invaded Us!!! They DO...,"[AmericaFIRST, Christian, NODACA]","[, user, , user, people, invaded, us, , , , be..."
3,"stop JUDGING bitches by there cover, jus cuz s...",[],"[stop, judging, bitches, cover, , jus, cuz, ba..."
4,how about i knock heads off and send them gift...,[],"[knock, heads, send, gift, wrapped, moms, hous..."
...,...,...,...
2966,Silly Killary WANNABE !! And @user numbers JUS...,"[MAGA, MASA, BuildThatWall, BuildTheFuture]","[silly, killary, wannabe, , , , user, numbers,..."
2967,@user @user @user @user @user @user @user @use...,"[IllegalImmigrants, VoteBank]","[, user, , user, , user, , user, , user, , use..."
2968,@user StopImmigration,[],"[, user, stopimmigration]"
2969,That bitch on the spoiled whore list Thought...,[],"[bitch, spoiled, whore, list, thought, could, ..."


**Removing unwanted spaces**

In [58]:
Hate_Train_text_df["clean"]=Hate_Train_text_df["clean"].apply(lambda x: [y for y in x if y.strip() != '' and len(y.strip())>2])
Hate_Test_text_df["clean"]=Hate_Test_text_df["clean"].apply(lambda x: [y for y in x if y.strip() != '' and len(y.strip())>2])

In [59]:
Hate_Test_text_df

,0,hashtags,clean
0,"@user , you are correct that Reid certainly is...","[MAGA, KAG, POTUS, Trump, NEWS, VoteRed, NoDAC...","[user, correct, reid, certainly, weasel, sadly..."
1,Whoever just unfollowed me you a bitch,[],"[whoever, unfollowed, bitch]"
2,@user @user Those People Invaded Us!!! They DO...,"[AmericaFIRST, Christian, NODACA]","[user, user, people, invaded, beling, rights, ..."
3,"stop JUDGING bitches by there cover, jus cuz s...",[],"[stop, judging, bitches, cover, jus, cuz, bad,..."
4,how about i knock heads off and send them gift...,[],"[knock, heads, send, gift, wrapped, moms, hous..."
...,...,...,...
2966,Silly Killary WANNABE !! And @user numbers JUS...,"[MAGA, MASA, BuildThatWall, BuildTheFuture]","[silly, killary, wannabe, user, numbers, keep,..."
2967,@user @user @user @user @user @user @user @use...,"[IllegalImmigrants, VoteBank]","[user, user, user, user, user, user, user, use..."
2968,@user StopImmigration,[],"[user, stopimmigration]"
2969,That bitch on the spoiled whore list Thought...,[],"[bitch, spoiled, whore, list, thought, could, ..."


***OFFENSIVE DATASET***

In [60]:
MODEL = f"cardiffnlp/twitter-roberta-base-offensive"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [63]:
Off_Test_labels = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_labels.txt"
with urllib.request.urlopen(Off_Test_labels) as f:   
    html_1 = f.read().decode('utf-8').split("\n")
    
Off_Test_label_df=pd.DataFrame(html_1)
print(Off_Test_label_df)

Off_Test_text = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_text.txt"
with urllib.request.urlopen(Off_Test_text) as f:   
    html_2 = f.read().decode('utf-8').split("\n")
       
Off_Test_text_df=pd.DataFrame(html_2)
print(Off_Test_text_df)

     0
0    1
1    0
2    0
3    0
4    0
..  ..
856  0
857  1
858  0
859  0
860   

[861 rows x 1 columns]
                                                     0
0    #ibelieveblaseyford is liar she is fat ugly li...
1    @user @user @user I got in a pretty deep debat...
2    ...if you want more shootings and more death, ...
3    Angels now have 6 runs. Five of them have come...
4    #Travel #Movies and Unix #Fortune combined  Vi...
..                                                 ...
856  @user @user @user @user @user @user @user @use...
857  #Conservatives don’t care what you post..it’s ...
858  #antifa #Resist.. Trump is trying to bring wor...
859  #Maine you need to face facts @user doesn’t re...
860                                                   

[861 rows x 1 columns]


In [64]:
Off_Train_labels = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_labels.txt"
with urllib.request.urlopen(Off_Train_labels) as f:   
    html_1 = f.read().decode('utf-8').split("\n")
    
Off_Train_labels_df=pd.DataFrame(html_1)
print(Off_Train_labels_df)

Off_Train_text = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_text.txt"
with urllib.request.urlopen(Off_Train_text) as f:   
    html_2 = f.read().decode('utf-8').split("\n")
       
Off_Train_text_df=pd.DataFrame(html_2)
print(Off_Train_text_df)

       0
0      0
1      1
2      0
3      0
4      1
...   ..
11912  0
11913  0
11914  0
11915  1
11916   

[11917 rows x 1 columns]
                                                       0
0      @user Bono... who cares. Soon people will unde...
1      @user Eight years the republicans denied obama...
2      @user Get him some line help. He is gonna be j...
3                   @user @user She is great. Hi Fiona! 
4      @user She has become a parody unto herself? Sh...
...                                                  ...
11912  @user Do we dare say he is... better than Nyjer? 
11913                    @user No idea who he is. Sorry 
11914  #Professor Who Shot Self Over Trump Says Gun C...
11915  @user @user @user Here your proof!  Our Africa...
11916                                                   

[11917 rows x 1 columns]


In [68]:
#extract hashtags and put them in new column named hashtag 
Off_Train_text_df["hashtags"]=Off_Train_text_df[0].apply(lambda x:re.findall(r"#(\w+)",x))

Off_Test_text_df["hashtags"]=Off_Test_text_df[0].apply(lambda x:re.findall(r"#(\w+)",x))

#translate emojis in training
Off_Train_text_df["clean"]=Off_Train_text_df[0].apply(lambda x: emoji.demojize(x))

Off_Test_text_df["clean"]=Off_Test_text_df[0].apply(lambda x: emoji.demojize(x))

#remove urls 
Off_Train_text_df["clean"]=Off_Train_text_df["clean"].apply(lambda x: re.sub(r"http:\S+",'',x))

Off_Test_text_df["clean"]=Off_Test_text_df["clean"].apply(lambda x: re.sub(r"http:\S+",'',x))

#tokenize  tweets
Off_Train_text_df["clean"]=Off_Train_text_df["clean"].apply(lambda x: nltk.word_tokenize(str(x).lower()))

Off_Test_text_df["clean"]=Off_Test_text_df["clean"].apply(lambda x: nltk.word_tokenize(str(x).lower()))

#remove stopwords and punctuations
stop_words = set(stopwords.words('english'))
Off_Train_text_df["clean"]=Off_Train_text_df["clean"].apply(lambda x: [y for y in x if (y not in stop_words)])
Off_Train_text_df["clean"]=Off_Train_text_df["clean"].apply(lambda x: [re.sub(r'['+string.punctuation+']','',y) for y in x])
Off_Train_text_df["clean"]=Off_Train_text_df["clean"].apply(lambda x: [re.sub('\\n','',y) for y in x])

Off_Test_text_df["clean"]=Off_Test_text_df["clean"].apply(lambda x: [y for y in x if (y not in stop_words)])
Off_Test_text_df["clean"]=Off_Test_text_df["clean"].apply(lambda x: [re.sub(r'['+string.punctuation+']','',y) for y in x])
Off_Test_text_df["clean"]=Off_Test_text_df["clean"].apply(lambda x: [re.sub('\\n','',y) for y in x])

#remove unwanted space
Off_Train_text_df["clean"]=Off_Train_text_df["clean"].apply(lambda x: [y for y in x if y.strip() != '' and len(y.strip())>2])
Off_Test_text_df["clean"]=Off_Test_text_df["clean"].apply(lambda x: [y for y in x if y.strip() != '' and len(y.strip())>2])

In [69]:
Off_Train_text_df

,0,hashtags,clean
0,@user Bono... who cares. Soon people will unde...,[],"[user, bono, cares, soon, people, understand, ..."
1,@user Eight years the republicans denied obama...,[],"[user, eight, years, republicans, denied, obam..."
2,@user Get him some line help. He is gonna be j...,[],"[user, get, line, help, gon, fine, game, went,..."
3,@user @user She is great. Hi Fiona!,[],"[user, user, great, fiona]"
4,@user She has become a parody unto herself? Sh...,[],"[user, become, parody, unto, certainly, taken,..."
...,...,...,...
11912,@user Do we dare say he is... better than Nyjer?,[],"[user, dare, say, better, nyjer]"
11913,@user No idea who he is. Sorry,[],"[user, idea, sorry]"
11914,#Professor Who Shot Self Over Trump Says Gun C...,"[Professor, AR]","[professor, shot, self, trump, says, gun, cont..."
11915,@user @user @user Here your proof! Our Africa...,[],"[user, user, user, proof, african, unitedstate..."


***IRONY DATASET***

In [70]:
MODEL = f"cardiffnlp/twitter-roberta-base-irony"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [71]:
Irony_Train_labels = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/train_labels.txt"
with urllib.request.urlopen(Irony_Train_labels) as f:   
    html_1 = f.read().decode('utf-8').split("\n")
    
Irony_Train_labels_df=pd.DataFrame(html_1)
print(Irony_Train_labels_df)

Irony_Train_text = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/train_text.txt"
with urllib.request.urlopen(Irony_Train_text) as f:   
    html_2 = f.read().decode('utf-8').split("\n")
       
Irony_Train_text_df=pd.DataFrame(html_2)
print(Irony_Train_text_df)

      0
0     1
1     0
2     1
3     1
4     1
...  ..
2858  1
2859  0
2860  0
2861  1
2862   

[2863 rows x 1 columns]
                                                      0
0     seeing ppl walking w/ crutches makes me really...
1     look for the girl with the broken smile, ask h...
2     Now I remember why I buy books online @user #s...
3     @user @user So is he banded from wearing the c...
4     Just found out there are Etch A Sketch apps.  ...
...                                                 ...
2858  Women getting hit on by married managers at @u...
2859  @user no but i followed you and i saw you post...
2860  @user I dont know what it is but I'm in love y...
2861  @user @user @user For having union representat...
2862                                                   

[2863 rows x 1 columns]


In [76]:
Irony_Test_labels = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/test_labels.txt"
with urllib.request.urlopen(Irony_Test_labels) as f:   
    html_1 = f.read().decode('utf-8').split("\n")
    
Irony_Test_labels_df=pd.DataFrame(html_1)
print(Irony_Test_labels_df)

Irony_Test_text = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/test_text.txt"
with urllib.request.urlopen(Irony_Test_text) as f:   
    html_2 = f.read().decode('utf-8').split("\n")
       
Irony_Test_text_df=pd.DataFrame(html_2)
print(Irony_Test_text_df)

     0
0    0
1    1
2    0
3    0
4    1
..  ..
780  0
781  0
782  1
783  0
784   

[785 rows x 1 columns]
                                                     0
0    @user Can U Help?||More conservatives needed o...
1    Just walked in to #Starbucks and asked for a "...
2                                      #NOT GONNA WIN 
3    @user He is exactly that sort of person. Weirdo! 
4    So much #sarcasm at work mate 10/10 #boring 10...
..                                                 ...
780  Congrats to my fav @user & her team & my birth...
781  @user Jessica sheds tears at her fan signing e...
782  #Irony: al jazeera is pro Anti - #GamerGate be...
783  #NOT ALL 👌 There good & bad in every occupatio...
784                                                   

[785 rows x 1 columns]


In [77]:
#extract hashtags and put them in new column named hashtag 
Irony_Train_text_df["hashtags"]=Irony_Train_text_df[0].apply(lambda x:re.findall(r"#(\w+)",x))

Irony_Test_text_df["hashtags"]=Irony_Test_text_df[0].apply(lambda x:re.findall(r"#(\w+)",x))

#translate emojis in training
Irony_Train_text_df["clean"]=Irony_Train_text_df[0].apply(lambda x: emoji.demojize(x))

Irony_Test_text_df["clean"]=Irony_Test_text_df[0].apply(lambda x: emoji.demojize(x))

#remove urls 
Irony_Train_text_df["clean"]=Irony_Train_text_df["clean"].apply(lambda x: re.sub(r"http:\S+",'',x))

Irony_Test_text_df["clean"]=Irony_Test_text_df["clean"].apply(lambda x: re.sub(r"http:\S+",'',x))

#tokenize  tweets
Irony_Train_text_df["clean"]=Irony_Train_text_df["clean"].apply(lambda x: nltk.word_tokenize(str(x).lower()))

Irony_Test_text_df["clean"]=Irony_Test_text_df["clean"].apply(lambda x: nltk.word_tokenize(str(x).lower()))

#remove stopwords and punctuations
stop_words = set(stopwords.words('english'))
Irony_Train_text_df["clean"]=Irony_Train_text_df["clean"].apply(lambda x: [y for y in x if (y not in stop_words)])
Irony_Train_text_df["clean"]=Irony_Train_text_df["clean"].apply(lambda x: [re.sub(r'['+string.punctuation+']','',y) for y in x])
Irony_Train_text_df["clean"]=Irony_Train_text_df["clean"].apply(lambda x: [re.sub('\\n','',y) for y in x])

Irony_Test_text_df["clean"]=Irony_Test_text_df["clean"].apply(lambda x: [y for y in x if (y not in stop_words)])
Irony_Test_text_df["clean"]=Irony_Test_text_df["clean"].apply(lambda x: [re.sub(r'['+string.punctuation+']','',y) for y in x])
Irony_Test_text_df["clean"]=Irony_Test_text_df["clean"].apply(lambda x: [re.sub('\\n','',y) for y in x])

#remove unwanted space
Irony_Train_text_df["clean"]=Irony_Train_text_df["clean"].apply(lambda x: [y for y in x if y.strip() != '' and len(y.strip())>2])
Irony_Test_text_df["clean"]=Irony_Test_text_df["clean"].apply(lambda x: [y for y in x if y.strip() != '' and len(y.strip())>2])

In [78]:
Irony_Test_text_df


,0,hashtags,clean
0,@user Can U Help?||More conservatives needed o...,[TSU],"[user, help, more, conservatives, needed, tsu,..."
1,"Just walked in to #Starbucks and asked for a ""...","[Starbucks, irony]","[walked, starbucks, asked, tall, blonde, hahah..."
2,#NOT GONNA WIN,[NOT],"[gon, win]"
3,@user He is exactly that sort of person. Weirdo!,[],"[user, exactly, sort, person, weirdo]"
4,So much #sarcasm at work mate 10/10 #boring 10...,"[sarcasm, boring, dead, shit, sleeping, sarcasm]","[much, sarcasm, work, mate, 1010, boring, 100,..."
...,...,...,...
780,Congrats to my fav @user & her team & my birth...,"[OhioState, OREvsOSU]","[congrats, fav, user, team, birthplace, team, ..."
781,@user Jessica sheds tears at her fan signing e...,[],"[user, jessica, sheds, tears, fan, signing, ev..."
782,#Irony: al jazeera is pro Anti - #GamerGate be...,"[Irony, GamerGate]","[irony, jazeera, pro, anti, gamergate, feminis..."
783,#NOT ALL 👌 There good & bad in every occupatio...,"[NOT, Stop, Not]","[okhand, good, bad, every, occupation, race, b..."


**Model**

In [5]:
op=[]
#len(test_labels)
for i in range(10):
  text = Test_text_df[0][i]
  text = preprocess(text)
#print("txt:",text,"\n")

  encoded_input = tokenizer(text, return_tensors='pt')
#print("encoded_input:",encoded_input,"\n")

  output = model(**encoded_input)
#print("op:",output,"\n")

  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
#print("Max score:",max(scores),"\n")

#index(max(scores))
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  op.append(ranking[0])
  #print("\nRanking:",ranking[0])

#for i in range(scores.shape[0]):
  l = labels[ranking[0]]
  s = scores[ranking[0]]
#print(l,np.round(float(s), 4))

In [6]:
op

[1, 10, 6, 1, 1, 17, 2, 10, 12, 10]